In [ ]:
%run src/Packets.ipynb
%run src/Agent.ipynb

Import libraries

In [ ]:
import sys
sys.path.append(".")
import numpy as np

read the configurations file

In [ ]:
# from configparser import ConfigParser
  
# configur = ConfigParser()
# import builtins
# configur.read(builtins.current_filename)

# #configur.read('config.ini')

Iot generates packages.
The IotNodes class is initialised with - 
- rate - rate of package genration
- def_ttl - default ttl of each new package
- position - its own position in the grid or matrix
- neighbours - list of all of its neighbours
- total_packets - numbers of total packets generated by the Node.
- queue - list of packets in the IotNode

In [ ]:
## def_ttl initial value of ttl
class IotNodes():

    def __init__(self, rate, def_ttl, x, y, transmission_rate = 10):
        self.rate = rate
        self.def_ttl = def_ttl
        self.position = (x,y)
        self.neighbours = []
        self.total_packets = 0
        self.queue = []  

generatePacket
- each time the function is called, create new packets equal to the rate
- increase the value of total packets
- create each packet with def_ttl and append it in the queue.

In [ ]:
class IotNodes(IotNodes):    
    def generatePacket(self):   # TODO what is this ? , and use of numpy ? and use of random ? 
        """
        generates 0-rate number of packets and adds to the back of queue
        In every cycle, transmission_rate number of packets would be transmitted to agents
        """
        l=[]
        #print(self.rate)
        #num_packets = np.random.randint(low = 1,high = self.rate) # TODO : why random ?
        num_packets = self.rate
        self.total_packets += num_packets
        for i in range(num_packets):
            self.queue.append(packet(self.def_ttl))

findNeighbour
- returns the neighbour with the least size queue.
- if the neighbour is a Base Station, return the base station.
- all nodes are agents, UAV, Base and IotNode
- agent.getVal() returns the queue length of the agent.
- return the agent with the minimum queue size.

In [ ]:
class IotNodes(IotNodes):       
    def findNeighbour(self):
        ## TODO :policy to find the neighbour : returning one with min queue size as of now
        ## IF base station is neigh
        for agent in self.neighbours:
            if(agent.isBase()):
                return agent
        queues = np.array([agent.getVal() for agent in self.neighbours])
        agents = [agent for agent in self.neighbours]
        return agents[ np.argmin(queues) ]

isUAV, isBase, isBaseStation, isBlock - False \
isIot - True

In [ ]:
class IotNodes(IotNodes):   
    def isUAV(self):
        return False
      
    def isBase(self):
        return False
     
    def isIot(self):
        return True

    def isBaseStation(self):
        return False

    def isBlock(self):
        return False

run
- In each run, we generate packets.
- if there are no neighbours, then there is no point in proceeding
- if there are neighbours and we have packets, the send the packet at the head of the queue and send it to the neighbour with the smallest queue, and make the neighbour to accept the packet.

In [ ]:
class IotNodes(IotNodes):   
    def run(self):
        # self.total_packets+=self.rate
        self.generatePacket()
        # for packet in self.queue:
        #     packet.decrease_ttl()
        for i in range(self.rate):
            if len(self.neighbours)==0:
                continue
            if self.getQueueSize() > 0:   # check if queue has any packets
                packet = self.queue.pop(0)
                agent=self.findNeighbour()
                #print("Node at",self.getPosition(),"to",agent.getPosition())
                agent.acceptPacket(packet)

- getPosition - return the node position
- addNeighbour - append a neighbour to the list of neighbours
- getVal - return the number of total packets stored till now
- getQueueSize - return the length of the queue.

In [ ]:
class IotNodes(IotNodes):   
    def getPosition(self):
        return self.position

    def addNeighbour(self,neighbour: Agent):
        self.neighbours.append(neighbour)

    def getVal(self):               # returns the total number of packets generated
        return self.total_packets

    def getQueueSize(self):
        return len(self.queue)

reset the number of total_packets and the queue

In [ ]:
class IotNodes(IotNodes):   
    def reset(self):
        """
        reset everything in the agent to turn on test mode
        """
        self.total_packets=0
        self.queue = []